In [1]:
using MPSKit, MPSKitModels, TensorKit, TensorOperations

In [7]:
function bosonicOperators(cutoff::Int, elt=Float64)
    creadat = zeros(elt, cutoff + 1, cutoff + 1)

    for i in 1:cutoff
        creadat[i + 1, i] = sqrt(i)
    end

    a⁺ = TensorMap(creadat, ℝ^(cutoff + 1), ℝ^(cutoff + 1))
    a⁻ = TensorMap(collect(creadat'), ℝ^(cutoff + 1), ℝ^(cutoff + 1))
    return (a⁺, a⁻)
end

bosonicOperators (generic function with 2 methods)

In [8]:
# define Hamiltionian for infinite basis with chemical potential
function BoseHubbardHamiltonian(t, U, μ, L, M)

    a⁺, a⁻ = bosonicOperators(M) # bosonic operators
    n_op = a⁺ * a⁻

    H_i = TensorMap(zeros, Float64, (ℝ^(M+1))^2 ← (ℝ^(M+1))^2)
    H_i += -t * (a⁺ ⊗ a⁻ + a⁻ ⊗ a⁺) + U/2 * n_op * (n_op - id(ℝ^(M+1))) ⊗ id(ℝ^(M+1))
    H_i += -μ * n_op ⊗ id(ℝ^(M+1))

    return @mpoham sum(H_i{i,i+1} for i in (1:L))
end

BoseHubbardHamiltonian (generic function with 1 method)

In [9]:
H = BoseHubbardHamiltonian(0.1, 2, 0, 12, 5)

MPOHamiltonian{CartesianSpace, TensorMap{CartesianSpace, 2, 2, Trivial, Matrix{Float64}, Nothing, Nothing}, Float64}(MPSKit.SparseMPOSlice{CartesianSpace, TensorMap{CartesianSpace, 2, 2, Trivial, Matrix{Float64}, Nothing, Nothing}, Float64}[[TensorMap((ℝ^1 ⊗ ℝ^6) ← (ℝ^6 ⊗ ℝ^1)):
[:, :, 1, 1] =
 1.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.0  1.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 0.0  0.0  1.0  0.0  0.0  0.0

[:, :, 4, 1] =
 0.0  0.0  0.0  1.0  0.0  0.0

[:, :, 5, 1] =
 0.0  0.0  0.0  0.0  1.0  0.0

[:, :, 6, 1] =
 0.0  0.0  0.0  0.0  0.0  1.0
 TensorMap((ℝ^1 ⊗ ℝ^6) ← (ℝ^6 ⊗ ℝ^3)):
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 0.0  0.0  -0.08276058886023684  0.0  0.0  0.0

[:, :, 4, 1] =
 0.0  0.0  0.0  -0.24828176658071027  0.0  0.0

[:, :, 5, 1] =
 0.0  0.0  0.0  0.0  -0.4965635331614207  0.0

[:, :, 6, 1] =
 0.0  0.0  0.0  0.0  0.0  -0.8276058886023683

[:, :, 1, 2] =
 -2.867980228534952e-17  5.647435290633063e-20  0

In [10]:
state = FiniteMPS(rand, Float64, 12, ℝ^(5+1), ℝ^20)

FiniteMPS{TensorMap{CartesianSpace, 2, 1, Trivial, Matrix{Float64}, Nothing, Nothing}, TensorMap{CartesianSpace, 1, 1, Trivial, Matrix{Float64}, Nothing, Nothing}}(Union{Missing, TensorMap{CartesianSpace, 2, 1, Trivial, Matrix{Float64}, Nothing, Nothing}}[TensorMap((ℝ^1 ⊗ ℝ^6) ← ProductSpace(ℝ^6)):
[:, :, 1] =
 0.652704285315959  0.060890130576106476  …  0.25885590971933786

[:, :, 2] =
 0.04482796148521204  0.6047024318722769  …  -0.16081244687411608

[:, :, 3] =
 -0.21278982593794782  -0.034538050160690933  …  0.8727822369337828

[:, :, 4] =
 -0.6496854149444002  -0.11927122071909456  …  -0.1935852527841051

[:, :, 5] =
 0.06903843320299866  0.5140580806121774  …  -0.1590205817712813

[:, :, 6] =
 -0.3159598330895224  0.5924129890901936  …  0.28743929151080416
, TensorMap((ℝ^6 ⊗ ℝ^6) ← ProductSpace(ℝ^20)):
[:, :, 1] =
  0.3786548412602646     0.14121277552045797   …  0.44770731045411577
  0.27523627519476607    0.1055956642418196       0.271416896084748
  0.20663681890517366    0.121

In [11]:
(groundstate,_) = find_groundstate(state, H, DMRG(tol=1e-10, maxiter=200));

┌ Info: Iteraton 0 error 0.15265875134852655
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 1 error 0.002631997465173779
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 2 error 9.742684090112332e-5
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 3 error 3.004993962705486e-5
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 4 error 1.1698278463513535e-5
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


...

┌ Info: Iteraton 128 error 1.4504171625645153e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 129 error 1.3348259873342677e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 130 error 1.2284718458959904e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27
┌ Info: Iteraton 131 error 1.1305816216474824e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 132 error 1.0404801832435933e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 133 error 9.575581288900007e-11
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


In [12]:
E0 = sum(expectation_value(groundstate, H))

-0.9239709943340368

In [14]:
a⁺, a⁻ = bosonicOperators(5) # bosonic operators
n_op = a⁺ * a⁻

TensorMap(ProductSpace(ℝ^6) ← ProductSpace(ℝ^6)):
 0.0  0.0  0.0                 0.0                 0.0  0.0
 0.0  1.0  0.0                 0.0                 0.0  0.0
 0.0  0.0  2.0000000000000004  0.0                 0.0  0.0
 0.0  0.0  0.0                 2.9999999999999996  0.0  0.0
 0.0  0.0  0.0                 0.0                 4.0  0.0
 0.0  0.0  0.0                 0.0                 0.0  5.000000000000001


In [17]:
N_b = sum(expectation_value(groundstate, n_op))

9.999999986715075

In [18]:
N_b = sum(expectation_value(state, n_op))

29.89949946615446